<a href="https://colab.research.google.com/github/YashSunidhi/2015/blob/master/transformer_multilabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [1]:
!pip install transformers
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
import pickle
from transformers import *
from tqdm import tqdm, trange
from ast import literal_eval

     |████████████████████████████████| 778kB 9.0MB/s 
     |████████████████████████████████| 890kB 24.2MB/s 
     |████████████████████████████████| 3.0MB 34.2MB/s 
     |████████████████████████████████| 1.1MB 46.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=869743b73a1e63ac8c0c38ce97a571a7d4a4a27bca253ecf81de9f54e8cc29fa
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Using TensorFlow backend.


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

## Load and Preprocess Training Data

Dataset will be tokenized then split into training and validation sets. The validation set will be used to monitor training. For testing a separate test set will be loaded for analysis.

In [5]:
df = pd.read_csv(r"/content/vaccine_bx.csv",encoding = 'ISO-8859-1')
df.head()

,Entry Name,Type of Entry,Country,Date,Month,EE - Level of Expertise,HCP Specialty,Role,Text Description,Topic,CONTROL,RWE UK Program,RWE Quebec,Positive Data,Negative Data,Data supports value of VAX/prescription,Data supports inclusion in NIP,Data does not support inclusion in NIP,Safety data is relevant,Reactions to Impact data,Impact data exceeds expectations,Impact data in line with expectations,Reaction to Effectiveness data,Effectiveness data is acceptable,Effectiveness data is disappointing,Impact better indicator than effectiveness,Data Difficulty to understand,Understand difference impact vs effectiveness,Reactions to lack of herd immunity,Lack of herd immunity is acceptable,Lack of herd immunity is a concern,Uncertainty on booster dosage,Need for local RWE programs,Reactions to Australian Carriage Data,MATS concept,More data needed,May Label Update,Asking for articles / education materials,Raising the need for education (Patients/parents/HCPs),Safety / AE,Thermostability/ Cold chain breach,Co-adminstration,Administration site/route,Dosing schedule/ Intervals between dosages,2+1 vs. 3+1 schedule,Duration of immunity / immuno persistence,Administration after expiration/ open package,Access / Reimbursement,Specific patient populations,Extend vaccination to other cohorts,Bexsero vs. Trumenba,Vaccine content / ingredients,Other,COVID impact?
0,KMI000006679_EU_PL,mVOC,Poland,4/7/2020,4,Non-EE,Immunology,EE,"According to EE, phe data is relevant and supp...",Guidelines / Disease State,4,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,KMI000018186_BIGEU_ES,mVOC,Spain,4/1/2020,4,Non-EE,NaN,Specialist,"Data from UK are apparently robust, so we need...",Guidelines / Disease State,3,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KMI000018187_BIGEU_ES,mVOC,Spain,4/1/2020,4,Non-EE,NaN,Specialist,In Italy there are effectiveness data with Bex...,Other,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KMI000018188_BIGEU_ES,mVOC,Spain,4/1/2020,4,Non-EE,NaN,Payor,Cross protection to other serogroups could it ...,Other,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,KMI000018189_BIGEU_ES,mVOC,Spain,4/1/2020,4,Non-EE,NaN,Payor,If we can appreciate an impact to serogroups W...,Guidelines / Disease State,4,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [6]:
test_df = df[900:]
df = df[:900]
# test_df = pd.read_csv('test.csv')
# test_labels_df = pd.read_csv('test_labels.csv')
# test_df = test_df.merge(test_labels_df, on='id', how='left')
# test_label_cols = list(test_df.columns[2:])
# print('Null values: ', test_df.isnull().values.any()) #should not be any null sentences or labels
# print('Same columns between train and test: ', label_cols == test_label_cols) #columns should be the same
test_df.head()

,Entry Name,Type of Entry,Country,Date,Month,EE - Level of Expertise,HCP Specialty,Role,Text Description,Topic,CONTROL,RWE UK Program,RWE Quebec,Positive Data,Negative Data,Data supports value of VAX/prescription,Data supports inclusion in NIP,Data does not support inclusion in NIP,Safety data is relevant,Reactions to Impact data,Impact data exceeds expectations,Impact data in line with expectations,Reaction to Effectiveness data,Effectiveness data is acceptable,Effectiveness data is disappointing,Impact better indicator than effectiveness,Data Difficulty to understand,Understand difference impact vs effectiveness,Reactions to lack of herd immunity,Lack of herd immunity is acceptable,Lack of herd immunity is a concern,Uncertainty on booster dosage,Need for local RWE programs,Reactions to Australian Carriage Data,MATS concept,More data needed,May Label Update,Asking for articles / education materials,Raising the need for education (Patients/parents/HCPs),Safety / AE,Thermostability/ Cold chain breach,Co-adminstration,Administration site/route,Dosing schedule/ Intervals between dosages,2+1 vs. 3+1 schedule,Duration of immunity / immuno persistence,Administration after expiration/ open package,Access / Reimbursement,Specific patient populations,Extend vaccination to other cohorts,Bexsero vs. Trumenba,Vaccine content / ingredients,Other,COVID impact?
900,MI-296371_LATAM_BR,MI,Brazil,5/29/2020,5,EE - Local,Pediatrics,NaN,"LADHANI, SN. et al. Vaccination of infants wit...",NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
901,MI-296372_LATAM_BR,MI,Brazil,5/29/2020,5,EE - Local,Infectious Disease,NaN,"LADHANI, SN. et al. Vaccination of infants wit...",NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
902,MI-296373_LATAM_BR,MI,Brazil,5/29/2020,5,EE - National,Pediatrics,NaN,"LADHANI, SN. et al. Vaccination of infants wit...",NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
903,MI-296374_LATAM_BR,MI,Brazil,5/29/2020,5,EE - National,Pediatrics,NaN,Nolan et al. Antibody persistence and booster ...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
904,MI-296377_LATAM_BR,MI,Brazil,5/29/2020,5,Non-EE,Pediatrics,NaN,"LADHANI, SN. et al. Vaccination of infants wit...",NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
print('Unique comments: ', df['Text Description'].nunique() == df.shape[0])
print('Null values: ', df.isnull().values.any())
# df[df.isna().any(axis=1)]

Unique comments:  False
Null values:  True


In [8]:
print('average sentence length: ', df['Text Description'].str.split().str.len().mean())
print('stdev sentence length: ', df['Text Description'].str.split().str.len().std())

average sentence length:  30.9
stdev sentence length:  34.9835333900657


In [32]:
print('average sentence length: ', test_df['Text Description'].str.split().str.len().mean())
print('stdev sentence length: ', test_df['Text Description'].str.split().str.len().std())

average sentence length:  32.494623655913976
stdev sentence length:  47.95699056354196


In [30]:
test_df['string_length'] = test_df['Text Description'].str.split().str.len()

In [31]:
test_df.to_csv('test.csv')

In [9]:
columns = [ 'RWE UK Program', 'RWE Quebec', 'Positive Data',
       'Negative Data', 'Data supports value of VAX/prescription',
       'Data supports inclusion in NIP',
       'Data does not support inclusion in NIP', 'Safety data is relevant',
       'Reactions to Impact data', 'Impact data exceeds expectations',
       'Impact data in line with expectations',
       'Reaction to Effectiveness data', 'Effectiveness data is acceptable',
       'Effectiveness data is disappointing',
       'Impact better indicator than effectiveness',
       'Data Difficulty to understand',
       'Understand difference impact vs effectiveness',
       'Reactions to lack of herd immunity',
       'Lack of herd immunity is acceptable',
       'Lack of herd immunity is a concern', 'Uncertainty on booster dosage',
       'Need for local RWE programs', 'Reactions to Australian Carriage Data',
       'MATS concept', 'More data needed', 'May Label Update',
       'Asking for articles / education materials',
       'Raising the need for education (Patients/parents/HCPs)', 'Safety / AE',
       'Thermostability/ Cold chain breach', 'Co-adminstration',
       'Administration site/route',
       'Dosing schedule/ Intervals between dosages', '2+1 vs. 3+1 schedule',
       'Duration of immunity / immuno persistence',
       'Administration after expiration/ open package',
       'Access / Reimbursement', 'Specific patient populations',
       'Extend vaccination to other cohorts', 'Bexsero vs. Trumenba',
       'Vaccine content / ingredients', 'Other', 'COVID impact?']

In [10]:
#cols = df.columns
label_cols = columns
num_labels = len(label_cols)
print('Label columns: ', label_cols)

Label columns:  ['RWE UK Program', 'RWE Quebec', 'Positive Data', 'Negative Data', 'Data supports value of VAX/prescription', 'Data supports inclusion in NIP', 'Data does not support inclusion in NIP', 'Safety data is relevant', 'Reactions to Impact data', 'Impact data exceeds expectations', 'Impact data in line with expectations', 'Reaction to Effectiveness data', 'Effectiveness data is acceptable', 'Effectiveness data is disappointing', 'Impact better indicator than effectiveness', 'Data Difficulty to understand', 'Understand difference impact vs effectiveness', 'Reactions to lack of herd immunity', 'Lack of herd immunity is acceptable', 'Lack of herd immunity is a concern', 'Uncertainty on booster dosage', 'Need for local RWE programs', 'Reactions to Australian Carriage Data', 'MATS concept', 'More data needed', 'May Label Update', 'Asking for articles / education materials', 'Raising the need for education (Patients/parents/HCPs)', 'Safety / AE', 'Thermostability/ Cold chain breach

In [11]:
print('Count of 1 per label: \n', df[label_cols].sum(), '\n') # Label counts, may need to downsample or upsample
print('Count of 0 per label: \n', df[label_cols].eq(0).sum())

Count of 1 per label: 
 RWE UK Program                                            221.0
RWE Quebec                                                  7.0
Positive Data                                             185.0
Negative Data                                               1.0
Data supports value of VAX/prescription                   112.0
Data supports inclusion in NIP                             18.0
Data does not support inclusion in NIP                      2.0
Safety data is relevant                                    33.0
Reactions to Impact data                                   91.0
Impact data exceeds expectations                            0.0
Impact data in line with expectations                       9.0
Reaction to Effectiveness data                             55.0
Effectiveness data is acceptable                           16.0
Effectiveness data is disappointing                        12.0
Impact better indicator than effectiveness                 32.0
Data Difficulty 

In [12]:
df = df.sample(frac=1).reset_index(drop=True) #shuffle rows

In [13]:
df = df.fillna(0)

In [14]:
df['one_hot_labels'] = list(df[label_cols].values)
df.head()

,Entry Name,Type of Entry,Country,Date,Month,EE - Level of Expertise,HCP Specialty,Role,Text Description,Topic,CONTROL,RWE UK Program,RWE Quebec,Positive Data,Negative Data,Data supports value of VAX/prescription,Data supports inclusion in NIP,Data does not support inclusion in NIP,Safety data is relevant,Reactions to Impact data,Impact data exceeds expectations,Impact data in line with expectations,Reaction to Effectiveness data,Effectiveness data is acceptable,Effectiveness data is disappointing,Impact better indicator than effectiveness,Data Difficulty to understand,Understand difference impact vs effectiveness,Reactions to lack of herd immunity,Lack of herd immunity is acceptable,Lack of herd immunity is a concern,Uncertainty on booster dosage,Need for local RWE programs,Reactions to Australian Carriage Data,MATS concept,More data needed,May Label Update,Asking for articles / education materials,Raising the need for education (Patients/parents/HCPs),Safety / AE,Thermostability/ Cold chain breach,Co-adminstration,Administration site/route,Dosing schedule/ Intervals between dosages,2+1 vs. 3+1 schedule,Duration of immunity / immuno persistence,Administration after expiration/ open package,Access / Reimbursement,Specific patient populations,Extend vaccination to other cohorts,Bexsero vs. Trumenba,Vaccine content / ingredients,Other,COVID impact?,one_hot_labels
0,MI-43190_APNA_AU,MI,Australia,3/20/2020,3,Non-EE,Nursing,0,*GSK Consult Bexsero Recommendations for re va...,Access / Reimbursement,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,MI-295590_LATAM_BR,MI,Brazil,2/17/2020,2,Non-EE,Other,0,Bexsero Dosage after dose delay,Formulation / Packaging / Dosing,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,KMI000007677_APNA_NZ,mVOC,New Zealand,3/4/2020,3,Non-EE,Pediatrics,EE,"Meningococcal B 4CMenB programme, consider tar...",Guidelines / Disease State,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,KMI000017315_BIGEU_IT,mVOC,Italy,2/19/2020,2,Non-EE,Public Health and General Preventive Med,Other,I believe that the UK data confirms that the c...,0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,KMI000016450_BIGEU_FR,mVOC,France,1/10/2020,1,EE - Regional,Pediatrics,EE,Discussion around English data and the impact ...,Research,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [15]:
labels = list(df.one_hot_labels.values)
comments = list(df['Text Description'].values)

Load the pretrained tokenizer that corresponds to your choice in model. e.g.,

```
BERT:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) 

XLNet:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=False) 

RoBERTa:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=False)
```


In order to avoid memory issues with Google Colab, I enforce a max_length of 100 tokens. Note that some sentences may not adequately represent each label because of this.

In [16]:
max_length = 100
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # tokenizer
encodings = tokenizer.batch_encode_plus(comments,max_length=max_length,pad_to_max_length=True) # tokenizer's encoding method
print('tokenizer outputs: ', encodings.keys())

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



tokenizer outputs:  dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [17]:
input_ids = encodings['input_ids'] # tokenized and encoded sentences
token_type_ids = encodings['token_type_ids'] # token type ids
attention_masks = encodings['attention_mask'] # attention masks

In [18]:
# Identifying indices of 'one_hot_labels' entries that only occur once - this will allow us to stratify split our training data later
label_counts = df.one_hot_labels.astype(str).value_counts()
one_freq = label_counts[label_counts==1].keys()
one_freq_idxs = sorted(list(df[df.one_hot_labels.astype(str).isin(one_freq)].index), reverse=True)
print('df label indices with only one instance: ', one_freq_idxs)

df label indices with only one instance:  [896, 892, 884, 868, 857, 856, 852, 847, 832, 815, 801, 780, 777, 776, 775, 762, 751, 736, 727, 714, 705, 700, 699, 695, 690, 684, 682, 679, 670, 669, 666, 664, 654, 651, 645, 637, 635, 633, 630, 624, 616, 613, 603, 596, 594, 592, 566, 562, 555, 545, 539, 514, 513, 506, 500, 498, 492, 477, 475, 471, 469, 468, 465, 460, 450, 442, 430, 422, 404, 403, 401, 398, 396, 390, 383, 376, 375, 373, 368, 356, 350, 345, 338, 336, 325, 315, 312, 309, 286, 283, 274, 270, 263, 260, 258, 257, 247, 236, 235, 224, 220, 215, 209, 197, 184, 180, 178, 173, 164, 148, 147, 143, 134, 132, 125, 124, 122, 115, 100, 99, 98, 97, 96, 90, 88, 74, 62, 59, 56, 50, 49, 40, 36, 33, 29, 26, 25, 17, 3]


In [19]:
# Gathering single instance inputs to force into the training set after stratified split
one_freq_input_ids = [input_ids.pop(i) for i in one_freq_idxs]
one_freq_token_types = [token_type_ids.pop(i) for i in one_freq_idxs]
one_freq_attention_masks = [attention_masks.pop(i) for i in one_freq_idxs]
one_freq_labels = [labels.pop(i) for i in one_freq_idxs]

Be sure to handle all classes during validation using "stratify" during train/validation split:

In [20]:
# Use train_test_split to split our data into train and validation sets

train_inputs, validation_inputs, train_labels, validation_labels, train_token_types, validation_token_types, train_masks, validation_masks = train_test_split(input_ids, labels, token_type_ids,attention_masks,
                                                            random_state=2020, test_size=0.10, stratify = labels)

# Add one frequency data to train data
train_inputs.extend(one_freq_input_ids)
train_labels.extend(one_freq_labels)
train_masks.extend(one_freq_attention_masks)
train_token_types.extend(one_freq_token_types)

# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_token_types = torch.tensor(train_token_types)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)
validation_token_types = torch.tensor(validation_token_types)

In [21]:
# Select a batch size for training. For fine-tuning with XLNet, the authors recommend a batch size of 32, 48, or 128. We will use 32 here to avoid memory issues.
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels, train_token_types)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels, validation_token_types)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [22]:
torch.save(validation_dataloader,'validation_data_loader')
torch.save(train_dataloader,'train_data_loader')

## Load Model & Set Params

Load the appropriate model below, each model already contains a single dense layer for classification on top.



```
BERT:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

XLNet:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=num_labels)

RoBERTa:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)
```



In [23]:
# Load model, the pretrained model will include a single linear classification layer on top for classification. 
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

Setting custom optimization parameters for the AdamW optimizer https://huggingface.co/transformers/main_classes/optimizer_schedules.html

In [24]:
# setting custom optimization parameters. You may implement a scheduler here as well.
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [25]:
optimizer = AdamW(optimizer_grouped_parameters,lr=2e-5,correct_bias=True)
# optimizer = AdamW(model.parameters(),lr=2e-5)  # Default optimization

## Train Model

In [26]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 30

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):

  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()

  # Tracking variables
  tr_loss = 0 #running loss
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()

    # # Forward pass for multiclass classification
    # outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    # loss = outputs[0]
    # logits = outputs[1]

    # Forward pass for multilabel classification
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]
    loss_func = BCEWithLogitsLoss() 
    loss = loss_func(logits.view(-1,num_labels),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    # loss_func = BCELoss() 
    # loss = loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    train_loss_set.append(loss.item())    

    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    # scheduler.step()
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))

###############################################################################

  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Variables to gather full output
  logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

  # Predict
  for i, batch in enumerate(validation_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    with torch.no_grad():
      # Forward pass
      outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      b_logit_pred = outs[0]
      pred_label = torch.sigmoid(b_logit_pred)

      b_logit_pred = b_logit_pred.detach().cpu().numpy()
      pred_label = pred_label.to('cpu').numpy()
      b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

  # Flatten outputs
  pred_labels = [item for sublist in pred_labels for item in sublist]
  true_labels = [item for sublist in true_labels for item in sublist]

  # Calculate Accuracy
  threshold = 0.50
  pred_bools = [pl>threshold for pl in pred_labels]
  true_bools = [tl==1 for tl in true_labels]
  val_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')*100
  val_flat_accuracy = accuracy_score(true_bools, pred_bools)*100

  print('F1 Validation Accuracy: ', val_f1_accuracy)
  print('Flat Validation Accuracy: ', val_flat_accuracy)

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Train loss: 0.5718972511016406


Epoch:   3%|▎         | 1/30 [00:08<03:56,  8.15s/it]

F1 Validation Accuracy:  0.0
Flat Validation Accuracy:  0.0
Train loss: 0.3894410374072882


Epoch:   7%|▋         | 2/30 [00:16<03:45,  8.07s/it]

...

F1 Validation Accuracy:  70.73170731707317
Flat Validation Accuracy:  50.649350649350644
Train loss: 0.08124890350378476


Epoch:  97%|█████████▋| 29/30 [03:49<00:07,  7.89s/it]

F1 Validation Accuracy:  74.69879518072288
Flat Validation Accuracy:  54.54545454545454
Train loss: 0.07882310908574325


Epoch: 100%|██████████| 30/30 [03:56<00:00,  7.90s/it]

F1 Validation Accuracy:  74.25149700598801
Flat Validation Accuracy:  55.84415584415584


In [27]:
torch.save(model.state_dict(), 'bert_model_vaccine')

## Load and Preprocess Test Data

In [ ]:
#test_df = test_df[~test_df[test_label_cols].eq(-1).any(axis=1)] #remove irrelevant rows/comments with -1 values
test_df = test_df.fillna(0)
test_df['one_hot_labels'] = list(test_df[columns].values)
test_df.head()

,Entry Name,Type of Entry,Country,Date,Month,EE - Level of Expertise,HCP Specialty,Role,Text Description,Topic,CONTROL,RWE UK Program,RWE Quebec,Positive Data,Negative Data,Data supports value of VAX/prescription,Data supports inclusion in NIP,Data does not support inclusion in NIP,Safety data is relevant,Reactions to Impact data,Impact data exceeds expectations,Impact data in line with expectations,Reaction to Effectiveness data,Effectiveness data is acceptable,Effectiveness data is disappointing,Impact better indicator than effectiveness,Data Difficulty to understand,Understand difference impact vs effectiveness,Reactions to lack of herd immunity,Lack of herd immunity is acceptable,Lack of herd immunity is a concern,Uncertainty on booster dosage,Need for local RWE programs,Reactions to Australian Carriage Data,MATS concept,More data needed,May Label Update,Asking for articles / education materials,Raising the need for education (Patients/parents/HCPs),Safety / AE,Thermostability/ Cold chain breach,Co-adminstration,Administration site/route,Dosing schedule/ Intervals between dosages,2+1 vs. 3+1 schedule,Duration of immunity / immuno persistence,Administration after expiration/ open package,Access / Reimbursement,Specific patient populations,Extend vaccination to other cohorts,Bexsero vs. Trumenba,Vaccine content / ingredients,Other,COVID impact?,one_hot_labels
900,MI-296371_LATAM_BR,MI,Brazil,5/29/2020,5,EE - Local,Pediatrics,0,"LADHANI, SN. et al. Vaccination of infants wit...",0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
901,MI-296372_LATAM_BR,MI,Brazil,5/29/2020,5,EE - Local,Infectious Disease,0,"LADHANI, SN. et al. Vaccination of infants wit...",0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
902,MI-296373_LATAM_BR,MI,Brazil,5/29/2020,5,EE - National,Pediatrics,0,"LADHANI, SN. et al. Vaccination of infants wit...",0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
903,MI-296374_LATAM_BR,MI,Brazil,5/29/2020,5,EE - National,Pediatrics,0,Nolan et al. Antibody persistence and booster ...,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
904,MI-296377_LATAM_BR,MI,Brazil,5/29/2020,5,Non-EE,Pediatrics,0,"LADHANI, SN. et al. Vaccination of infants wit...",0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
# Gathering input data
test_labels = list(test_df.one_hot_labels.values)
test_comments = list(test_df['Text Description'].values)

In [ ]:
# Encoding input data
test_encodings = tokenizer.batch_encode_plus(test_comments,max_length=max_length,pad_to_max_length=True)
test_input_ids = test_encodings['input_ids']
test_token_type_ids = test_encodings['token_type_ids']
test_attention_masks = test_encodings['attention_mask']

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
# Make tensors out of data
test_inputs = torch.tensor(test_input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_attention_masks)
test_token_types = torch.tensor(test_token_type_ids)
# Create test dataloader
test_data = TensorDataset(test_inputs, test_masks, test_labels, test_token_types)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
# Save test dataloader
torch.save(test_dataloader,'test_data_loader')

## Prediction and Metics

In [ ]:
# Test

# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

#track variables
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

# Predict
for i, batch in enumerate(test_dataloader):
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels, b_token_types = batch
  with torch.no_grad():
    # Forward pass
    outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    b_logit_pred = outs[0]
    pred_label = torch.sigmoid(b_logit_pred)

    b_logit_pred = b_logit_pred.detach().cpu().numpy()
    pred_label = pred_label.to('cpu').numpy()
    b_labels = b_labels.to('cpu').numpy()

  tokenized_texts.append(b_input_ids)
  logit_preds.append(b_logit_pred)
  true_labels.append(b_labels)
  pred_labels.append(pred_label)

# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]
pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]
# Converting flattened binary values to boolean values
true_bools = [tl==1 for tl in true_labels]

We need to threshold our sigmoid function outputs which range from [0, 1]. Below I use 0.50 as a threshold.

In [ ]:
pred_bools = [pl>0.80 for pl in pred_labels] #boolean output after thresholding

# Print and save classification report
print('Test F1 Accuracy: ', f1_score(true_bools, pred_bools,average='micro'))
print('Test Flat Accuracy: ', accuracy_score(true_bools, pred_bools),'\n')
clf_report = classification_report(true_bools,pred_bools,target_names=columns)
pickle.dump(clf_report, open('classification_report.txt','wb')) #save report
print(clf_report)

Test F1 Accuracy:  0.4602272727272727
Test Flat Accuracy:  0.3225806451612903 

                                                        precision    recall  f1-score   support

                                        RWE UK Program       0.72      0.62      0.67        21
                                            RWE Quebec       0.00      0.00      0.00         1
                                         Positive Data       0.82      0.64      0.72        14
                                         Negative Data       0.00      0.00      0.00         0
               Data supports value of VAX/prescription       0.00      0.00      0.00        10
                        Data supports inclusion in NIP       0.00      0.00      0.00         7
                Data does not support inclusion in NIP       0.00      0.00      0.00         1
                               Safety data is relevant       0.00      0.00      0.00         6
                              Reactions to Impact data 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1

In [ ]:
len(columns)

43

## Output Dataframe

In [ ]:
idx2label = dict(zip(range(43),columns))
print(idx2label)

{0: 'RWE UK Program', 1: 'RWE Quebec', 2: 'Positive Data', 3: 'Negative Data', 4: 'Data supports value of VAX/prescription', 5: 'Data supports inclusion in NIP', 6: 'Data does not support inclusion in NIP', 7: 'Safety data is relevant', 8: 'Reactions to Impact data', 9: 'Impact data exceeds expectations', 10: 'Impact data in line with expectations', 11: 'Reaction to Effectiveness data', 12: 'Effectiveness data is acceptable', 13: 'Effectiveness data is disappointing', 14: 'Impact better indicator than effectiveness', 15: 'Data Difficulty to understand', 16: 'Understand difference impact vs effectiveness', 17: 'Reactions to lack of herd immunity', 18: 'Lack of herd immunity is acceptable', 19: 'Lack of herd immunity is a concern', 20: 'Uncertainty on booster dosage', 21: 'Need for local RWE programs', 22: 'Reactions to Australian Carriage Data', 23: 'MATS concept', 24: 'More data needed', 25: 'May Label Update', 26: 'Asking for articles / education materials', 27: 'Raising the need for 

In [ ]:
# Getting indices of where boolean one hot vector true_bools is True so we can use idx2label to gather label names
true_label_idxs, pred_label_idxs=[],[]
for vals in true_bools:
  true_label_idxs.append(np.where(vals)[0].flatten().tolist())
for vals in pred_bools:
  pred_label_idxs.append(np.where(vals)[0].flatten().tolist())

In [ ]:
# Gathering vectors of label names using idx2label
true_label_texts, pred_label_texts = [], []
for vals in true_label_idxs:
  if vals:
    true_label_texts.append([idx2label[val] for val in vals])
  else:
    true_label_texts.append(vals)

for vals in pred_label_idxs:
  if vals:
    pred_label_texts.append([idx2label[val] for val in vals])
  else:
    pred_label_texts.append(vals)

In [ ]:
# Decoding input ids to comment text
comment_texts = [tokenizer.decode(text,skip_special_tokens=True,clean_up_tokenization_spaces=False) for text in tokenized_texts]

In [ ]:
# Converting lists to df
comparisons_df = pd.DataFrame({'comment_text': comment_texts, 'true_labels': true_label_texts, 'pred_labels':pred_label_texts})
comparisons_df.to_csv('comparisons.csv')
comparisons_df.head()

,comment_text,true_labels,pred_labels
0,"ladhani , sn . et al . vaccination of infants ...",[Asking for articles / education materials],[Asking for articles / education materials]
1,"ladhani , sn . et al . vaccination of infants ...",[Asking for articles / education materials],[Asking for articles / education materials]
2,"ladhani , sn . et al . vaccination of infants ...",[Asking for articles / education materials],[Asking for articles / education materials]
3,nolan et al . antibody persistence and booster...,[Duration of immunity / immuno persistence],[]
4,"ladhani , sn . et al . vaccination of infants ...",[Asking for articles / education materials],[Asking for articles / education materials]


## Bonus - Optimizing threshold value for micro F1 score

Doing this may result in a trade offs between precision, flat accuracy and micro F1 accuracy. You may tune the threshold however you want.

In [ ]:
# Calculate Accuracy - maximize F1 accuracy by tuning threshold values. First with 'macro_thresholds' on the order of e^-1 then with 'micro_thresholds' on the order of e^-2

macro_thresholds = np.array(range(1,10))/10

f1_results, flat_acc_results = [], []
for th in macro_thresholds:
  pred_bools = [pl>th for pl in pred_labels]
  test_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')
  test_flat_accuracy = accuracy_score(true_bools, pred_bools)
  f1_results.append(test_f1_accuracy)
  flat_acc_results.append(test_flat_accuracy)

best_macro_th = macro_thresholds[np.argmax(f1_results)] #best macro threshold value

micro_thresholds = (np.array(range(10))/100)+best_macro_th #calculating micro threshold values

f1_results, flat_acc_results = [], []
for th in micro_thresholds:
  pred_bools = [pl>th for pl in pred_labels]
  test_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')
  test_flat_accuracy = accuracy_score(true_bools, pred_bools)
  f1_results.append(test_f1_accuracy)
  flat_acc_results.append(test_flat_accuracy)

best_f1_idx = np.argmax(f1_results) #best threshold value

# Printing and saving classification report
print('Best Threshold: ', micro_thresholds[best_f1_idx])
print('Test F1 Accuracy: ', f1_results[best_f1_idx])
print('Test Flat Accuracy: ', flat_acc_results[best_f1_idx], '\n')

best_pred_bools = [pl>micro_thresholds[best_f1_idx] for pl in pred_labels]
clf_report_optimized = classification_report(true_bools,best_pred_bools, target_names=columns)
pickle.dump(clf_report_optimized, open('classification_report_optimized.txt','wb'))
print(clf_report_optimized)

Best Threshold:  0.51
Test F1 Accuracy:  0.5827505827505828
Test Flat Accuracy:  0.45161290322580644 

                                                        precision    recall  f1-score   support

                                        RWE UK Program       0.52      0.76      0.62        21
                                            RWE Quebec       0.00      0.00      0.00         1
                                         Positive Data       0.63      0.86      0.73        14
                                         Negative Data       0.00      0.00      0.00         0
               Data supports value of VAX/prescription       0.64      0.70      0.67        10
                        Data supports inclusion in NIP       0.00      0.00      0.00         7
                Data does not support inclusion in NIP       0.00      0.00      0.00         1
                               Safety data is relevant       0.00      0.00      0.00         6
                              Re

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1